In [1]:
!pip install faker


In [2]:
import sqlite3
from faker import Faker
import random
from datetime import datetime

faker = Faker('pt_BR')
conn = sqlite3.connect('livraria.db', timeout=30)
cursor = conn.cursor()

def criar_tabelas():
    cursor.executescript("""
    CREATE TABLE IF NOT EXISTS cliente (
        cod_cliente INTEGER PRIMARY KEY,
        nome TEXT,
        email TEXT,
        telefone TEXT
    );

    CREATE TABLE IF NOT EXISTS endereco (
        cod_endereco INTEGER PRIMARY KEY,
        cod_cliente INTEGER,
        rua TEXT,
        numero TEXT,
        cidade TEXT,
        estado TEXT,
        cep TEXT,
        FOREIGN KEY (cod_cliente) REFERENCES cliente(cod_cliente)
    );

    CREATE TABLE IF NOT EXISTS autor (
        cod_autor INTEGER PRIMARY KEY,
        nome TEXT,
        nacionalidade TEXT
    );

    CREATE TABLE IF NOT EXISTS editora (
        cod_editora INTEGER PRIMARY KEY,
        nome TEXT,
        contato TEXT
    );

    CREATE TABLE IF NOT EXISTS categoria (
        cod_categoria INTEGER PRIMARY KEY,
        nome TEXT
    );

    CREATE TABLE IF NOT EXISTS livro (
        cod_livro INTEGER PRIMARY KEY,
        titulo TEXT,
        cod_autor INTEGER,
        cod_editora INTEGER,
        cod_categoria INTEGER,
        ano_publicacao INTEGER,
        preco REAL,
        FOREIGN KEY (cod_autor) REFERENCES autor(cod_autor),
        FOREIGN KEY (cod_editora) REFERENCES editora(cod_editora),
        FOREIGN KEY (cod_categoria) REFERENCES categoria(cod_categoria)
    );

    CREATE TABLE IF NOT EXISTS estoque (
        cod_livro INTEGER PRIMARY KEY,
        quantidade INTEGER,
        FOREIGN KEY (cod_livro) REFERENCES livro(cod_livro)
    );

    CREATE TABLE IF NOT EXISTS pedido (
        cod_pedido INTEGER PRIMARY KEY,
        cod_cliente INTEGER,
        data_pedido TEXT,
        status TEXT,
        FOREIGN KEY (cod_cliente) REFERENCES cliente(cod_cliente)
    );

    CREATE TABLE IF NOT EXISTS tipo_pagamento (
        cod_tipo INTEGER PRIMARY KEY,
        nome TEXT
    );

    CREATE TABLE IF NOT EXISTS pagamento (
        cod_pagamento INTEGER PRIMARY KEY,
        cod_pedido INTEGER,
        cod_tipo INTEGER,
        data_pag TEXT,
        valor_pago REAL,
        FOREIGN KEY (cod_pedido) REFERENCES pedido(cod_pedido),
        FOREIGN KEY (cod_tipo) REFERENCES tipo_pagamento(cod_tipo)
    );

    CREATE TABLE IF NOT EXISTS item_pedido (
        cod_item INTEGER PRIMARY KEY,
        cod_pedido INTEGER,
        cod_livro INTEGER,
        quantidade INTEGER,
        preco_unitario REAL,
        FOREIGN KEY (cod_pedido) REFERENCES pedido(cod_pedido),
        FOREIGN KEY (cod_livro) REFERENCES livro(cod_livro)
    );
    """)
    conn.commit()

def insert_cliente(n):
    for i in range(0, n, 1000):
        for _ in range(min(1000, n - i)):
            cursor.execute("INSERT INTO cliente (nome, email, telefone) VALUES (?, ?, ?)", (
                faker.name(), faker.email(), faker.phone_number()))
        conn.commit()

def insert_endereco(n):
    ids = cursor.execute("SELECT cod_cliente FROM cliente").fetchall()
    for i in range(0, n, 1000):
        for _ in range(min(1000, n - i)):
            cod_cliente = random.choice(ids)[0]
            cursor.execute("INSERT INTO endereco (cod_cliente, rua, numero, cidade, estado, cep) VALUES (?, ?, ?, ?, ?, ?)", (
                cod_cliente, faker.street_name(), str(faker.building_number()),
                faker.city(), faker.estado_sigla(), faker.postcode()))
        conn.commit()

def insert_autor(n):
    for i in range(0, n, 1000):
        for _ in range(min(1000, n - i)):
            cursor.execute("INSERT INTO autor (nome, nacionalidade) VALUES (?, ?)", (
                faker.name(), faker.country()))
        conn.commit()

def insert_editora(n):
    for i in range(0, n, 1000):
        for _ in range(min(1000, n - i)):
            cursor.execute("INSERT INTO editora (nome, contato) VALUES (?, ?)", (
                faker.company(), faker.email()))
        conn.commit()

def insert_categoria(n):
    for i in range(0, n, 1000):
        for _ in range(min(1000, n - i)):
            cursor.execute("INSERT INTO categoria (nome) VALUES (?)", (
                faker.word().capitalize(),))
        conn.commit()

def insert_livro(n):
    autores = cursor.execute("SELECT cod_autor FROM autor").fetchall()
    editoras = cursor.execute("SELECT cod_editora FROM editora").fetchall()
    categorias = cursor.execute("SELECT cod_categoria FROM categoria").fetchall()


    for i in range(0, n, 1000):
        for _ in range(min(1000, n - i)):
            preco = round(random.uniform(25, 180), 2)
            cursor.execute("""
                INSERT INTO livro (titulo, cod_autor, cod_editora, cod_categoria, ano_publicacao, preco)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (
                faker.catch_phrase(),
                random.choice(autores)[0],
                random.choice(editoras)[0],
                random.choice(categorias)[0],
                random.randint(1990, 2024),
                preco
            ))
        conn.commit()


def insert_estoque(n):
    livros = cursor.execute("SELECT cod_livro FROM livro").fetchall()
    livros_unicos = list(set([l[0] for l in livros]))
    random.shuffle(livros_unicos)
    livros_usar = livros_unicos[:n]

    for i in range(0, len(livros_usar), 1000):
        for cod_livro in livros_usar[i:i+1000]:
            cursor.execute("INSERT INTO estoque (cod_livro, quantidade) VALUES (?, ?)", (
                cod_livro, random.randint(0, 150)))
        conn.commit()

def insert_pedido(n):
    clientes = cursor.execute("SELECT cod_cliente FROM cliente").fetchall()
    for i in range(0, n, 1000):
        for _ in range(min(1000, n - i)):
            cursor.execute("INSERT INTO pedido (cod_cliente, data_pedido, status) VALUES (?, ?, ?)", (
                random.choice(clientes)[0],
                faker.date_between(start_date='-4y', end_date='today').isoformat(),
                random.choice(['Pago', 'Pendente', 'Cancelado'])))
        conn.commit()

def insert_tipo_pagamento():
    tipos = ['Cartão', 'Boleto', 'Pix']
    for nome in tipos:
        cursor.execute("INSERT INTO tipo_pagamento (nome) VALUES (?)", (nome,))
    conn.commit()

def insert_item_pedido(n):
    pedidos = cursor.execute("SELECT cod_pedido FROM pedido").fetchall()
    livros = cursor.execute("SELECT cod_livro, preco FROM livro").fetchall()
    for i in range(0, n, 1000):
        for _ in range(min(1000, n - i)):
            cod_pedido = random.choice(pedidos)[0]
            cod_livro, preco = random.choice(livros)
            quantidade = random.randint(1, 5)
            preco = round(random.uniform(25, 180), 2)
            cursor.execute("""INSERT INTO item_pedido (cod_pedido, cod_livro, quantidade, preco_unitario)
                              VALUES (?, ?, ?, ?)""", (
                cod_pedido, cod_livro, quantidade, preco))
        conn.commit()

def insert_pagamento():
    pedidos = cursor.execute("SELECT cod_pedido, data_pedido FROM pedido").fetchall()
    tipos = cursor.execute("SELECT cod_tipo FROM tipo_pagamento").fetchall()

    for cod_pedido, data_pedido in pedidos:
        # Verifica valor real do pedido somando (preco_unitario * quantidade)
        cursor.execute("""
            SELECT SUM(preco_unitario * quantidade) FROM item_pedido WHERE cod_pedido = ?
        """, (cod_pedido,))
        resultado = cursor.fetchone()[0] or 0.0
        valor_pago = round(resultado, 2)

        # Só insere pagamento se valor for maior que zero
        if valor_pago > 0:
            data_pedido_date = datetime.strptime(data_pedido, '%Y-%m-%d').date()
            data_pag = faker.date_between(start_date=data_pedido_date, end_date='today').isoformat()
            cod_tipo = random.choice(tipos)[0]

            cursor.execute("""
                INSERT INTO pagamento (cod_pedido, cod_tipo, data_pag, valor_pago)
                VALUES (?, ?, ?, ?)""", (
                cod_pedido, cod_tipo, data_pag, valor_pago))

    conn.commit()


# Execução principal
criar_tabelas()
insert_cliente(25000)
insert_endereco(25000)
insert_autor(5000)
insert_editora(1000)
insert_categoria(100)
insert_livro(30000)
insert_estoque(10000)
insert_pedido(30000)
insert_tipo_pagamento()
insert_item_pedido(50000)
insert_pagamento()
conn.close()


In [3]:
import sqlite3
import pandas as pd

# Conecta ao banco de dados SQLite
conn = sqlite3.connect('livraria.db')

# Lista de tabelas
tabelas = ['cliente', 'endereco', 'autor', 'editora', 'livro', 'pedido', 'item_pedido', 'estoque', 'pagamento', 'categoria', 'tipo_pagamento']

# Exporta cada tabela para um CSV
for tabela in tabelas:
    df = pd.read_sql_query(f"SELECT * FROM {tabela}", conn)
    df.to_csv(f"{tabela}.csv", index=False)
    print(f"Tabela '{tabela}' exportada para '{tabela}.csv'.")

# Fecha a conexão
conn.close()

Tabela 'cliente' exportada para 'cliente.csv'.
Tabela 'endereco' exportada para 'endereco.csv'.
Tabela 'autor' exportada para 'autor.csv'.
Tabela 'editora' exportada para 'editora.csv'.
Tabela 'livro' exportada para 'livro.csv'.
Tabela 'pedido' exportada para 'pedido.csv'.
Tabela 'item_pedido' exportada para 'item_pedido.csv'.
Tabela 'estoque' exportada para 'estoque.csv'.
Tabela 'pagamento' exportada para 'pagamento.csv'.
Tabela 'categoria' exportada para 'categoria.csv'.
Tabela 'tipo_pagamento' exportada para 'tipo_pagamento.csv'.


In [4]:
import sqlite3
import pandas as pd
import os
from dotenv import load_dotenv
from azure.storage.filedatalake import DataLakeServiceClient
from azure.core.exceptions import ResourceExistsError

# Carregar variáveis de ambiente
load_dotenv()
account_name = os.getenv("ADLS_ACCOUNT_NAME")
file_system_name = os.getenv("ADLS_FILE_SYSTEM_NAME")
sas_token = os.getenv("ADLS_SAS_TOKEN")
landing_folder = "Csvs"

# Conectar ao Azure Data Lake
try:
    service_client = DataLakeServiceClient(
        account_url=f"https://{account_name}.dfs.core.windows.net",
        credential=sas_token
    )
    file_system_client = service_client.get_file_system_client(file_system_name)
    print("✅ Conectado ao ADLS com sucesso.")
except Exception as e:
    print("❌ Erro ao conectar ao ADLS:", e)
    raise e

✅ Conectado ao ADLS com sucesso.


In [5]:
# Criar diretório Landing/Csvs no ADLS (se não existir)

try:
    directory_client = file_system_client.get_directory_client(landing_folder)
    directory_client.create_directory()
    print(f"📁 Diretório '{landing_folder}' criado com sucesso.")
except ResourceExistsError:
    print(f"ℹ️ Diretório '{landing_folder}' já existe.")
except Exception as e:
    print(f"❌ Erro ao criar o diretório '{landing_folder}':", e)
    raise e

📁 Diretório 'Csvs' criado com sucesso.


In [6]:
# Fazer upload dos arquivos CSV para o Azure Data Lake

for tabela in tabelas:
    file_name = f"{tabela}.csv"
    try:
        file_client = directory_client.get_file_client(file_name)
        with open(file_name, "rb") as f:
            file_client.upload_data(f.read(), overwrite=True)
        print(f"✅ Upload de '{file_name}' para ADLS concluído.")
    except Exception as e:
        print(f"❌ Erro ao enviar '{file_name}' para o ADLS:", e)

✅ Upload de 'cliente.csv' para ADLS concluído.
✅ Upload de 'endereco.csv' para ADLS concluído.
✅ Upload de 'autor.csv' para ADLS concluído.
✅ Upload de 'editora.csv' para ADLS concluído.
✅ Upload de 'livro.csv' para ADLS concluído.
✅ Upload de 'pedido.csv' para ADLS concluído.
✅ Upload de 'item_pedido.csv' para ADLS concluído.
✅ Upload de 'estoque.csv' para ADLS concluído.
✅ Upload de 'pagamento.csv' para ADLS concluído.
✅ Upload de 'categoria.csv' para ADLS concluído.
✅ Upload de 'tipo_pagamento.csv' para ADLS concluído.
